In [16]:
import pandas as pd
import numpy as np
 

def compute_fliprate(df):
    # Flip rate is the accuracy with respect to the target labels
    # If the predicted sentiment is equal to the target we used to create the CF, then the label was flipped
    scores = []
    for i, row in df.iterrows():
        if row['Sentiment'] == row['Predicted_sentiment']:
            scores.append(1)
        else:
            scores.append(0)
    return np.mean(scores)


In [17]:
#LLM = 'mistral-20240118'

llms = ['llama2-20231209', 'mistral-20240118']
SPLIT = 'test'

results_dict = {}

MODELS_COL = 'model'
FLIPRATE_COL = 'cfs fliprate'
CFS_PPL_COL = 'cfs ppl'
ORIG_PPL_COL = 'orig ppl'
COUNT = '#instances'

for llm in llms: 
    df = pd.read_csv('../llms-ppl-preds/{}/sentiment/combined/{}.tsv'.format(llm, SPLIT), sep= '\t')
    #df['Predicted_sentiment'] = df['Predicted_sentiment'].map(lambda x: 'Positive' if str(x) == '1' else 'Negative')
    len_orig = len(pd.read_csv('../llms-ppl-preds/{}/sentiment/orig/{}.tsv'.format(llm, SPLIT), sep= '\t'))
    len_new = len(pd.read_csv('../llms-ppl-preds/{}/sentiment/new/{}.tsv'.format(llm, SPLIT), sep= '\t'))

    df_orig = df.iloc[:len_orig,:].reset_index()
    df_new = df.iloc[len_orig:,:].reset_index()

    assert(len(df_orig) == len_orig) 
    assert(len(df_new) == len_new) 
    df_orig.head()

    # LLM
    models = results_dict.get(MODELS_COL, [])
    models.append(llm.split('-')[0])
    results_dict[MODELS_COL] = models
    # CFs Fliprate
    cf_fliprate = compute_fliprate(df_new)
    cf_frates = results_dict.get(FLIPRATE_COL, [])
    cf_frates.append(cf_fliprate)
    results_dict[FLIPRATE_COL] = cf_frates

    # CFs PPL
    cf_ppl = df_new['perplexity'].mean()
    cf_ppls = results_dict.get(CFS_PPL_COL, [])
    cf_ppls.append(cf_ppl)
    results_dict[CFS_PPL_COL] = cf_ppls

    # ORIG PPL
    orig_ppl = df_orig['perplexity'].mean()
    orig_ppls = results_dict.get(ORIG_PPL_COL, [])
    orig_ppls.append(orig_ppl)
    results_dict[ORIG_PPL_COL] = orig_ppls

    # COUNT
    counts = results_dict.get(COUNT, [])
    counts.append(df_new['perplexity'].count())
    results_dict[COUNT] = counts

In [18]:
df_results = pd.DataFrame.from_dict(results_dict)
df_results['cfs fliprate'] = df_results['cfs fliprate']*100
df_results = df_results.round(2)
df_results.head()

,model,cfs fliprate,cfs ppl,orig ppl,#instances
0,llama2,79.92,46.99,43.69,483
1,mistral,94.16,48.54,43.97,411


In [ ]:
#SPLIT = 'train'
#df = pd.read_csv('../llms-ppl-preds/{}/sentiment/combined/{}.tsv'.format(LLM, SPLIT), sep= '\t')
#df['Predicted_sentiment'] = df['Predicted_sentiment'].map(lambda x: 'Positive' if str(x) == '1' else 'Negative')
#df.to_csv('../llms-ppl-preds/{}/sentiment/combined/{}.tsv'.format(LLM, SPLIT), sep= '\t', index= False)